In [5]:
import pandas as pd

In [2]:
paths_inputs = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/"
paths_outputs = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Indicadores/primeiros/"

In [6]:
nascidos_vivos = pd.read_csv(paths_inputs+"nascidos_vivos_tratados.csv")
nascidos_vivos["cod_municipio_6"] = nascidos_vivos["cod_municipio_6"].astype(str)

In [7]:
populacao = pd.read_parquet(paths_inputs+"populacao_tratados.parquet")
populacao = populacao[["cod_municipio", "pop_2021"]]

obitos = pd.read_parquet(paths_inputs + "obitos_tratados_microdados.parquet")
obitos_raw = pd.read_parquet(paths_inputs + "obitos_tratados.parquet")

dicio_cdi = pd.read_csv("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Dicionario, Documentanção e Codigos/dicionario_cid_10.csv")

#### Mortalidade infantil até 5 anos por 1K nascidos vivos (FEITO)

In [8]:
mask_obitos = obitos["causa"].str.startswith("B34")

In [12]:
populacao

,cod_municipio,pop_2021
0,1100015,22728
1,1100023,109523
2,1100031,5188
3,1100049,85893
4,1100056,16204
...,...,...
5565,5222005,13977
5566,5222054,8873
5567,5222203,6312
5568,5222302,5882


In [11]:
obitos_covid = obitos[mask_obitos].drop_duplicates("id")
obitos_covid = obitos_covid.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]

obitos_covid = obitos_covid.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"obitos_covid"})
obitos_covid["cod_municipio_6"] = obitos_covid["cod_municipio"].astype(str).apply(lambda x: x[:6])
obitos_covid.drop(columns=["cod_municipio"], inplace=True)

obitos_covid = populacao.merge(obitos_covid, on=["cod_municipio_6"], how="left")
obitos_covid["obitos_covid"] = obitos_covid["obitos_covid"].fillna(0)
obitos_covid["obitos_covid_1k"] = obitos_covid["obitos_covid"] / obitos_covid["pop_2021"] * 1000
obitos_covid.to_csv(paths_outputs+"obitos_covid.csv", index=False)


KeyError: 'cod_municipio_6'

In [15]:
#Mortalidade infantil até 5 anos por mil nascidos vivos
obitos_5 = obitos.query("idade < 5").groupby(["ano","id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]

obitos_5 = obitos_5.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"obitos_5_anos"})
obitos_5["cod_municipio_6"] = obitos_5["cod_municipio"].astype(str).apply(lambda x: x[:6])
obitos_5.drop(columns=["cod_municipio"], inplace=True)
obitos_5 = nascidos_vivos.merge(obitos_5, on=["cod_municipio_6"], how="left")
obitos_5["obitos_5_anos"] = obitos_5["obitos_5_anos"].fillna(0)
obitos_5["obitos_5_anos_1k"] = obitos_5["obitos_5_anos"] / obitos_5["nascidos_vivos"] * 1000

obitos_5.to_csv(paths_outputs+"obitos_ate_5_anos.csv", index=False)

#### Taxa de mortalidade da população devido à falta de ingestão de alimentos 100K população

In [2]:
obitos.groupby(["causa"]).sum().sort_values(by="idade", ascending=False).head(10)

In [75]:
def filter_group(
        group,
        number_of_digits,
        habitantes=False,
        n_habitantes=100000,
        n_nascidos_vivos=100000):
    dfs = []
    for n in range(0, number_of_digits+1):
        mask = obitos['causa'].str.startswith(f'{group}{n}')
        df = obitos[mask]
        dfs.append(df)
    dfs = pd.concat(dfs)
    dfs = dfs.drop_duplicates(subset="id")
    dfs = dfs[["id_municipio_ocorrencia", "causa"]]

    dfs = dfs.groupby("id_municipio_ocorrencia", as_index=False).count()
    dfs = dfs.sort_values(by="causa", ascending=False)
    dfs = dfs.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "causa":"obitos"})

    if habitantes:
        dfs = dfs.merge(populacao, on=["cod_municipio"], how="left")
        dfs["obitos"] = dfs["obitos"] / dfs["contagem"] * n_habitantes
        dfs = dfs[["cod_municipio", "obitos"]]
    else:
        dfs = dfs.merge(nascidos_vivos, on=["cod_municipio"], how="left")
        dfs["obitos"] = dfs["obitos"] / dfs["contagem"] * n_nascidos_vivos
        dfs = dfs[["cod_municipio", "obitos"]] 

    return dfs


In [76]:
df_e40_46 = filter_group(group="E4", number_of_digits=6)

In [78]:
df_e40_46.groupby("id_municipio_ocorrencia", as_index=False).count().sort_values(by="causa", ascending=False)

,id_municipio_ocorrencia,causa
96,1501402,577
1958,3550308,510
1677,3304557,412
1142,2927408,320
617,2507507,247
...,...,...
2254,4216701,1
2255,4217253,1
2256,4217501,1
652,2513802,1


In [5]:
dicio_cdi = pd.read_csv("C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/Dicionario, Documentanção e Codigos/dicionario_cid_10.csv")

In [10]:
dicio_cdi.query("sub_categoria == 'B342'")

,categoria,capitulo,descricao_categoria,grupo,sub_categoria,descricao_subcategoria,descricao_grupo,descricao_capitulo
506,B34,I,"Doenças por vírus, de localização não especifi...",B25-B34,B342,Infecção por coronavírus de localização não es...,Outras doenças por vírus,Algumas doenças infecciosas e parasitárias


#### Mortalidade materna 100K nascidos vivos (FEITO)

In [72]:
obitos = pd.read_parquet(paths_inputs+"obitos_tratados.parquet")

obitos_gravidez = obitos.query("obito_gravidez=='1' or obito_puerperio == '1'")
obitos_gravidez = obitos_gravidez.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]
obitos_gravidez = obitos_gravidez.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"obitos_gravidez"})
obitos_gravidez["cod_municipio_6"] = obitos_gravidez["cod_municipio"].astype(str).apply(lambda x: x[:6])
obitos_gravidez.drop(columns=["cod_municipio"], inplace=True)

obitos_gravidez = nascidos_vivos.merge(obitos_gravidez, on=["cod_municipio_6"], how="left")[["cod_municipio_6", "obitos_gravidez", "nascidos_vivos"]]

obitos_gravidez["obitos_gravidez"] = obitos_gravidez["obitos_gravidez"].fillna(0)
obitos_gravidez["obitos_gravidez_100k"] = obitos_gravidez["obitos_gravidez"]*100000/obitos_gravidez["nascidos_vivos"]
obitos_gravidez

obitos_gravidez.to_csv(paths_outputs+"obitos_gravidez.csv", index=False)

In [34]:
obitos.query("obito_puerperio == '1'")

,ano,sigla_uf,id_municipio_ocorrencia,idade,linha_a,linha_b,linha_c,linha_d,linha_ii,obito_gravidez,obito_puerperio,circunstancia_obito
241,2020,BA,2906501,48.0,*A419,*J960,*N179,*J189,*D649,2,1,None
507,2020,PB,2507507,37.0,*O141,*O152,None,None,None,2,1,None
794,2020,PE,2611606,30.0,*R571,*R58X,*O459,*O141,*O829*O601,2,1,None
1372,2020,MG,3118304,0.0,*P291,*P072,None,None,None,2,1,None
2235,2020,MA,2105302,18.0,*A419,*J189,*O909,None,None,2,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...
774812,2020,PB,2504009,18.0,*O994*I081*R570*I500,None,None,None,*O839,2,1,None
775217,2020,SE,2800308,35.0,*A419,*J189,*O985*B342*U071,None,*O829*I10X,2,1,None
777718,2020,SP,3537305,25.0,*O994,*O16X,*O159,None,None,2,1,None
777792,2020,PR,4106902,38.0,*R578,*O985*B342*U071*O141,*J988*U049,None,*O821*O249,2,1,None


In [44]:
269 + 272

541

In [60]:
puer = list(obitos.query("obito_puerperio=='1'").groupby(["id_municipio_ocorrencia"], as_index=False).count()["id_municipio_ocorrencia"])

In [63]:
gravi = obitos.query("obito_gravidez=='1'").groupby(["id_municipio_ocorrencia"], as_index=False).count()

In [66]:
gravi[gravi["id_municipio_ocorrencia"].isin(puer)]

,id_municipio_ocorrencia,ano,sigla_uf,idade,linha_a,linha_b,linha_c,linha_d,linha_ii,obito_gravidez,obito_puerperio,circunstancia_obito
2,1100205,1,1,1,1,1,1,0,0,1,1,0
5,1200401,1,1,1,1,1,1,0,1,1,1,0
9,1302603,5,5,5,5,4,3,3,1,5,5,1
13,1304062,1,1,1,1,1,1,0,0,1,1,1
14,1400100,3,3,3,3,2,3,2,1,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
261,5107925,1,1,1,0,0,1,0,0,1,1,1
264,5201108,2,2,2,2,2,0,0,0,2,2,1
265,5208707,13,13,13,12,11,8,4,7,13,13,0
269,5220108,1,1,1,1,1,0,0,0,1,1,0


In [68]:
272+269-109

432

In [69]:
obitos["obito_gravidez"] = obitos["obito_gravidez"].astype(str)
obitos["obito_puerperio"] = obitos["obito_puerperio"].astype(str)

obitos_gravidez = obitos.query("obito_gravidez=='1' or obito_puerperio == '1'")

obitos_gravidez = obitos_gravidez.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]
obitos_gravidez = obitos_gravidez.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"obitos_gravidez"})
obitos_gravidez

,cod_municipio,obitos_gravidez
0,1100015,1
1,1100106,1
2,1100122,1
3,1100205,6
4,1100288,1
...,...,...
427,5218904,1
428,5219704,1
429,5220108,2
430,5221858,2


#### Assassinatos de jovens

In [5]:
agg_sub_cat = dicio_cdi.query("grupo == 'X85-Y09'")["sub_categoria"].unique()
agg_cat = dicio_cdi.query("grupo == 'X85-Y09'")["categoria"].unique()
agg = list(agg_sub_cat) + list(agg_cat)

In [21]:
assassinatos = obitos[obitos["causa"].apply(lambda x: x in agg)]
assassinatos = assassinatos.drop_duplicates("id")

In [53]:
assasinatos_j = assassinatos.query("idade < 25")
assasinatos_j = assasinatos_j.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]
assasinatos_j = assasinatos_j.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"assasinatos_jovens"})
assasinatos_j = assasinatos_j.merge(populacao, on=["cod_municipio"], how="left")[["cod_municipio", "assasinatos_jovens", "pop_2021"]]
assasinatos_j["assasinatos_jovens_100k"] = assasinatos_j["assasinatos_jovens"]*100000/assasinatos_j["pop_2021"]
assasinatos_j.to_csv(paths_outputs+"assasinatos_jovens.csv", index=False)


##### Assasinatos Gerais

In [52]:
assasinatos_gerais = assassinatos.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]
assasinatos_gerais = assasinatos_gerais.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"assasinatos_gerais"})
assasinatos_gerais = assasinatos_gerais.merge(populacao, on=["cod_municipio"], how="left")[["cod_municipio", "assasinatos_gerais", "pop_2021"]]
assasinatos_gerais["assasinatos_gerais_100k"] = assasinatos_gerais["assasinatos_gerais"]*100000/assasinatos_gerais["pop_2021"]
assasinatos_gerais.to_csv(paths_outputs+"assasinatos_gerais.csv", index=False)

In [34]:
acidentes = []
for i in range(1, 10):
    string = "V0"+str(i)
    acidentes.append(string)
for i in range(10, 100):
    string = "V"+str(i)
    acidentes.append(string)

In [48]:
obitos_acidentes = obitos[obitos["causa"].apply(lambda x: x in acidentes)]
obitos_acidentes = obitos_acidentes.drop_duplicates("id")
obitos_acidentes = obitos_acidentes.groupby(["id_municipio_ocorrencia"], as_index=False).count()[["id_municipio_ocorrencia", "idade"]]
obitos_acidentes = obitos_acidentes.rename(columns={"id_municipio_ocorrencia":"cod_municipio", "idade":"obitos_acidentes"})
obitos_acidentes = obitos_acidentes.merge(populacao, on=["cod_municipio"], how="left")[["cod_municipio", "obitos_acidentes", "pop_2021"]]
obitos_acidentes["obitos_acidentes_100k"] = obitos_acidentes["obitos_acidentes"]*100000/obitos_acidentes["pop_2021"]

In [51]:
obitos_acidentes.to_csv(paths_outputs+"obitos_acidentes.csv", index=False)

acidentes